In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import plotly.express as px
from scipy import stats

In [ ]:
x = pd.read_csv("METADATA.csv")
x.head()

## A. Analyse descriptive

In [ ]:
x.describe()

In [ ]:
x['batch']=pd.Categorical(x['batch'],ordered=False)

### Commentaire :

* Transformation de la variable batch en variable qualitative.

In [ ]:
x.drop(['lat0','lon0','last_lon','last_lat'],axis = 1).hist(figsize=(14,14));

### Commentaires :

* La plupart des tortues pèsent entre 0 et 12 kg.

* Pour la taille, entre 10 et 40 cm

* Pour la durée de suivi, entre 0 et 500 jours.

* Distribution plus ou moins uniforme sur la distance totale parcourue. Etant donné que la durée de suivi varie, les distances sont plus ou moins longues.

* Distance moyenne journalière entre 15 et 30 km.

## B. Analyse multidimensionnelle

### Corrélation

In [ ]:
scatter_matrix(x.drop(['lat0', 'lon0', 'last_lon', 'last_lat'], axis=1), 
               figsize=(12, 12));

In [ ]:
plt.figure(figsize=(12,8))
corrMatrix = x.drop(['lat0', 'lon0', 'last_lon', 'last_lat'], axis=1).corr()
sns.heatmap(corrMatrix, cmap="vlag", vmin=-1, vmax=1);

#### Commentaires :

* Les variables de taille, poids et age des tortues sont fortement corrélés positivement : indice de corrélation proche de 1, corrélation linéaire.
* Des tortues d'une catégorie de taille spécifique gardent leurs capteurs beaucoup plus longtemps que d'autres.
* On ne peut rien dire des autres variables car l'indice de corrélation est trop faible, entre 0.25 et -0.25.

### Dispersion Taille et Statuts

In [ ]:
x['capture_status'].unique()

In [ ]:
print('Nb d\'individus avec Pound Net en capture status : ',
      len(x[x['capture_status']=='Pound Net']))

In [ ]:
# Fusion variables Batch et Capture - suppression label Pound Net en Light
f = lambda a : (a,'Light')[a == 'Pound Net']
x['capture_status'] = x['capture_status'].apply(f)

In [ ]:
x['capture_status'].unique()

In [ ]:
fig = px.box(x , y="SCL_cm", color="capture_status",width=700, height=350)
fig.show()

#### Commentaires

* Les tortues élevées en aquarium sont relachées en mer avec une taille beaucoup plus faible que celle repechées accidentellement en mer. 

* Piste : regarder vitesse de nage des aquariums VS capturées : on s'attend à ce que les tortues repechées parcourent plus de distance que celles élevées en aquarium, car elles sont plus grosses en taille.

In [ ]:
fig = px.box(x , y="daily_mean_distance (km)", color="capture_status",width=700, height=350)
fig.show()

#### Commentaires

* Contrairement à nos attentes, les médianes sont équivalentes quelque soit le mode de capture. 

* On remarque plus d'outliers sur les tortues elevées en aquarium.

* **Les variables daily_mean_distance & SCL ne sont pas correlées !!!**


Transformation en log de daily_mean et longevity ?

### Dispersion mois et distance

In [ ]:
def month_extract(x):
    sep1 = x.split('/')[0]
    sep = int(sep1)
    if sep==1: return 'janvier'
    elif sep==2: return 'fevrier'
    elif sep==3: return 'mars'
    elif sep==4: return 'avril'
    elif sep==5: return 'mai'
    elif sep==6: return 'juin'
    elif sep==7: return 'juillet'
    elif sep==8: return 'aout'
    elif sep==9: return 'septembre'
    elif sep==10: return 'octobre'
    elif sep==11: return 'novembre'
    elif sep==12: return 'decembre'

x['month'] = x['release'].apply(month_extract)

x['month'] = pd.Categorical(x['month'])

In [ ]:
fig = px.box(x, y="daily_mean_distance (km)", color="month",width=800, height=350,
             category_orders={'month': ['janvier','fevrier','mars','avril',
                                        'mai', 'juin', 'juillet', 'aout',
                                        'septembre', 'octobre', 'novembre',
                                        'decembre']})
fig.show()

In [ ]:
from collections import Counter
count = Counter(x['month']).most_common()
month_count = pd.DataFrame([list(m) for m in count], columns=['month', 'count'])
px.bar(month_count, x='month', y='count', title='Nombre de tortues lachées par mois')

#### Commentaires :

* Les tortues relachées aux mois d'avril et novembre parcourent de plus grandes distances.

* Le diagramme en batons ci dessus indique que les mois d'avril, mai, juillet, octobre et novembre sont les plus représentatifs du dataset.

* Piste : regrouper les tortues par saison et par batch.

### Saison et batch

In [ ]:
def season_var(x):
    if x=='janvier' or x=='fevrier' or x=='mars':
        return 'winter'
    elif x=='avril' or x=='mai' or x=='juin' :
        return 'spring'
    elif x=='juillet' or x=='aout' or x=='septembre' :
        return 'summer'
    else: 
        return 'autumn'
    
x['season'] = x['month'].apply(season_var)
x['season'] = pd.Categorical(x['season'])

In [ ]:
fig = px.box(x , y="daily_mean_distance (km)", color="season",width=800, height=350)
fig.show()

#### Commentaire :

* La variable saison ne nous permet pas d'analyser la distance parcourue par jour, nous allons nous concenter sur le batch et la longitude.

### Distance et batch

In [ ]:
fig = px.box(x, y="daily_mean_distance (km)", 
             color="batch",width=800, height=350)
fig.show()

#### Commentaires :

* Il existe une corrélation entre le batch et la distance parcourue en moyenne par jour.

* Batch 2, 3, 4, 5 VS le reste. 

* Piste : examiner la corrélation entre le batch et le mois de relache.

In [ ]:
mean_vitesse = x[x['batch']!=1]['daily_mean_distance (km)'].mean()
x['month'] = x['month'].astype(str)
fig = px.scatter(x[x['batch']!=1], x = 'batch',y = 'daily_mean_distance (km)',
                 color = 'month', title='Distance moyenne parcourue en fonction '+
                 'du mois de laché et du batch')
fig.update_layout(shapes=[
    dict(
      type= 'line',
      xref= 'paper', x0= 0, x1= 1,
      yref= 'y', y0= mean_vitesse, y1= mean_vitesse,
    )
])

#### Commentaires :

* Toutes les tortues captives sont relachées à partir du mois d'avril.

* Les tortues lachées en novembre (rouge) et avril (bleu marine) ont tendance à parcourir une distance supérieure à la moyenne par jour (cf droite noire). Cela correspond aux batch 2, 3, 4, 5. 

* C'est donc cohérent avec nos analyses précedentes. 


Piste : examiner le lien entre le courant et le mois de laché. (A VOIR DANS NOTEBOOK AVEC TOUTES LES TORTUES)

Lien entre longitude et batch ?

#### Correlation batch & longitude

In [ ]:
def eta_squared(x,y):
    moyenne_y = y.mean()
    classes = []
    for classe in x.unique():
        yi_classe = y[x==classe]
        classes.append({'ni': len(yi_classe),
                        'moyenne_classe': yi_classe.mean()})
    SCT = sum([(yj-moyenne_y)**2 for yj in y])
    SCE = sum([c['ni']*(c['moyenne_classe']-moyenne_y)**2 for c in classes])
    return SCE/SCT
    
eta_squared(x['batch'],x['lon0'])

#### Commentaires : 

* eta_squared = (variance interclasse) / (variance totale)

* Proche de 1 donc corrélation élevée entre la longitude et le batch.

In [ ]:
x0 = x[x['lon0']>0]
px.scatter(x0[x0['batch']!=1], x = 'lon0', y = 'daily_mean_distance (km)', 
           color = 'batch')

#### Commentaires :

* 1) Corrélation entre la longitude et le batch marquée importante. Des tortues de meme batch ont des longitudes de lachés similaires. (cf couleurs & longitude).


* 2) On observe que sur certaine longitude, la distance parcourue est plus ou moins grande (cf 140 VS 177). 
On suppose donc que la distance parcourue est influencée par la longitude.

Attention car l'échelle des longitudes est petite cependant dans la réalité les distances sont très grandes ...

* Dernier point : les batch 6 et 7 sont beaucoup plus lents ; cela confirme les hypothèses de la section précédente.

### Dispersion Taille & Poids

In [ ]:
fig = px.box(x , y="SCL_cm",width=300, height=500)
fig.show()

In [ ]:
fig = px.box(x , y="wt_kg", width=300, height=500)
fig.show()

In [ ]:
print('Pourcentage de tortues capturées accidentellement :', 
      (len(x.loc[x['capture_status']=='Deep']) + 
       len(x.loc[x['capture_status']=='Light'])) / len(x) * 100)

#### Commentaires  :

* Bcp d'outliers pour la taille : correspond aux tortues capturées car elles sont plus grosses que la moyenne et ces dernières représentent seulement 15% de la population.

* Etant donné que la taille et le poids sont corrélées positivement, on devrait retrouver autant d'outlier pour le poids. Or nous n'avons pas le poids des tortues capturés, d'où le fait qu'il y ait bcp moins d'outliers pour le boxplot du poids.

In [ ]:
# garder variable taille
# Modelisation composantes u et v ou normes et directions ? 